# Projeto Integrador 6 - Predição de Diabetes com Modelos de IA

## Grupo 04

# Import de Pacotes e Bibliotecas

In [1]:
!pip install optuna
!pip install ucimlrepo

In [23]:
import warnings
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import optuna
from plotly.io import show
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, FunctionTransformer, RobustScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import accuracy_score, f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, recall_score, precision_score
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from ucimlrepo import fetch_ucirepo
from functools import partial


warnings.filterwarnings('ignore') #Ignorar warnigs

# organizar os imports - explicar o motivo deles

# Load Dataset

In [3]:
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

data = cdc_diabetes_health_indicators.data.features
aux = cdc_diabetes_health_indicators.data.targets

data["Diabetes_binary"] = aux["Diabetes_binary"]

print(cdc_diabetes_health_indicators.metadata)

print(cdc_diabetes_health_indicators.variables)

data.columns

{'uci_id': 891, 'name': 'CDC Diabetes Health Indicators', 'repository_url': 'https://archive.ics.uci.edu/dataset/891/cdc+diabetes+health+indicators', 'data_url': 'https://archive.ics.uci.edu/static/public/891/data.csv', 'abstract': 'The Diabetes Health Indicators Dataset contains healthcare statistics and lifestyle survey information about people in general along with their diagnosis of diabetes. The 35 features consist of some demographics, lab test results, and answers to survey questions for each patient. The target variable for classification is whether a patient has diabetes, is pre-diabetic, or healthy. ', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Tabular', 'Multivariate'], 'num_instances': 253680, 'num_features': 21, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Sex', 'Age', 'Education Level', 'Income'], 'target_col': ['Diabetes_binary'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_

Index(['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke',
       'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income',
       'Diabetes_binary'],
      dtype='object')

# Pré-Processamento

## Transformação do data type para integer

In [4]:
data['Age']=data['Age'].astype('int64')
data['Education']=data['Education'].astype('int64')
data['Income']=data['Income'].astype('int64')
data['BMI']=data['BMI'].astype('int64')
data['CholCheck']=data['CholCheck'].astype('int64')
data['Smoker']=data['Smoker'].astype('int64')
data['Stroke']=data['Stroke'].astype('int64')
data['HeartDiseaseorAttack']=data['HeartDiseaseorAttack'].astype('int64')
data['PhysActivity']=data['PhysActivity'].astype('int64')
data['Fruits']=data['Fruits'].astype('int64')
data['Veggies']=data['Veggies'].astype('int64')
data['HvyAlcoholConsump']=data['HvyAlcoholConsump'].astype('int64')
data['AnyHealthcare']=data['AnyHealthcare'].astype('int64')
data['NoDocbcCost']=data['NoDocbcCost'].astype('int64')
data['GenHlth']=data['GenHlth'].astype('int64')
data['MentHlth']=data['MentHlth'].astype('int64')
data['PhysHlth']=data['PhysHlth'].astype('int64')
data['DiffWalk']=data['DiffWalk'].astype('int64')
data['Sex']=data['Sex'].astype('int64')
data['Diabetes_binary']=data['Diabetes_binary'].astype('int64')
data['HighBP']=data['HighBP'].astype('int64')
data['HighChol']=data['HighChol'].astype('int64')

## Remoção de duplicatas

In [5]:
data.duplicated().sum()

24206

In [7]:
data.drop_duplicates(inplace=True)

In [6]:
data.shape

(253680, 22)

In [9]:

data.head()

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,Diabetes_binary
0,1,1,1,40,1,0,0,0,0,1,...,0,5,18,15,1,0,9,4,3,0
1,0,0,0,25,1,0,0,1,0,0,...,1,3,0,0,0,0,7,6,1,0
2,1,1,1,28,0,0,0,0,1,0,...,1,5,30,30,1,0,9,4,8,0
3,1,0,1,27,0,0,0,1,1,1,...,0,2,0,0,0,0,11,3,6,0
4,1,1,1,24,0,0,0,1,1,1,...,0,2,3,0,0,0,11,5,4,0


## Divisão dos tipos de Variáveis

In [16]:
categorical_columns = ['GenHlth', 'Age', 'Education', 'Income']
numerical_columns = ['BMI', 'MentHlth', 'PhysHlth']
binary_columns = ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

# Divisão de Dados - Treino, Validação e Teste

Optamos por realizar o estudo do optuna com a base sampleada para economizar tempo, uma vez que cada tentativa de execução superava o tempo limite do colab.

In [17]:
data1 = data.sample(frac=0.05, random_state=42) # Base de dados sampleada

x = data1.drop('Diabetes_binary', axis = 1)
y = data1['Diabetes_binary']

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.15, random_state=42, stratify=y_train)


print("Treino\n",y_train.value_counts())
print("\nValidação \n",y_validation.value_counts())
print("\nTeste\n",y_test.value_counts())

Treino
 Diabetes_binary
0    7451
1    1173
Name: count, dtype: int64

Validação 
 Diabetes_binary
0    1316
1     207
Name: count, dtype: int64

Teste
 Diabetes_binary
0    2192
1     345
Name: count, dtype: int64


## PCA

In [14]:
pca = PCA(n_components=5)
pca.fit(x_train)
X_train = pca.fit_transform(x_train)
X_test = pca.fit_transform(x_test)

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

# Modelos

## KNN

In [19]:
preprocessor_knn = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns),
        ('bin', 'passthrough', binary_columns)
    ])

knn_pipeline = Pipeline([
    ('preprocessor', preprocessor_knn),
    ('classifier', KNeighborsClassifier())
])

In [20]:
def objective_knn(trial):
    pipeline = Pipeline([
        ('preprocessor', preprocessor_knn),
        ('knn', KNeighborsClassifier())
    ])
    k = trial.suggest_int('k', 2, 25)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])
    pipeline.set_params(knn__n_neighbors=k, knn__weights=weights, knn__metric=metric)
    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_validation)
    score = f1_score(y_validation, y_pred)
    return score




study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=100, n_jobs=-1)

study_knn.best_params

[I 2024-11-23 19:50:31,594] A new study created in memory with name: no-name-7b76c87c-7a4f-414d-8199-5e03647af8cd
[I 2024-11-23 19:50:31,928] Trial 1 finished with value: 0.24489795918367346 and parameters: {'k': 8, 'weights': 'distance', 'metric': 'minkowski'}. Best is trial 1 with value: 0.24489795918367346.
[I 2024-11-23 19:50:32,279] Trial 0 finished with value: 0.1532258064516129 and parameters: {'k': 17, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 1 with value: 0.24489795918367346.
[I 2024-11-23 19:50:32,592] Trial 2 finished with value: 0.1532258064516129 and parameters: {'k': 17, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 1 with value: 0.24489795918367346.
[I 2024-11-23 19:50:32,859] Trial 3 finished with value: 0.12903225806451613 and parameters: {'k': 8, 'weights': 'uniform', 'metric': 'euclidean'}. Best is trial 1 with value: 0.24489795918367346.
[I 2024-11-23 19:50:33,046] Trial 4 finished with value: 0.1532258064516129 and parameters: {'k':

{'k': 2, 'weights': 'distance', 'metric': 'minkowski'}

### Melhores Hiper-parâmetros k=2, weights=distance, metric=minkowski

In [ ]:
score = np.zeros(30)
acc = np.zeros(30)
rec = np.zeros(30)
prec = np.zeros(30)
score_train = np.zeros(30)
acc_train = np.zeros(30)
rec_train = np.zeros(30)
prec_train = np.zeros(30)

for i in range(30):
  df = data.sample(frac=1, random_state=42)
  x = df.drop(columns=['Diabetes_binary'])
  y = df['Diabetes_binary']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10, stratify=y)

  pipeline = Pipeline([
      ('preprocessor', preprocessor_knn),
      ('knn', KNeighborsClassifier())
  ])

  pipeline.set_params(knn__n_neighbors=2, knn__weights='distance', knn__metric='minkowski')
  pipeline.fit(x_train, y_train)
  y_train_pred = pipeline.predict(x_train)
  y_pred = pipeline.predict(x_test)
  score[i] = f1_score(y_test, y_pred)
  acc[i] = accuracy_score(y_test, y_pred)
  rec[i] = recall_score(y_test, y_pred)
  prec[i] = precision_score(y_test, y_pred)

  score_train[i] = f1_score(y_train, y_train_pred)
  acc_train[i] = accuracy_score(y_train, y_train_pred)
  rec_train[i] = recall_score(y_train, y_train_pred)
  prec_train[i] = precision_score(y_train, y_train_pred)


print("MÉTRICAS DE TREINO")
print("F1_SCORE: \n")
print(f" Média -> {score_train.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc_train.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec_train.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec_train.mean()}")
print(f" Desvio Padrão -> {prec_train.std()}")
print("\n\n---------------------\n\n")
print("MÉTRICAS DE TESTE")
print("F1_SCORE: \n")
print(f" Média -> {score.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec.mean()}")
print(f" Desvio Padrão -> {prec.std()}")

## Logistic Regression

In [21]:
preprocessor_lr = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns),
        ('bin', 'passthrough', binary_columns)
    ])

lr_pipeline = Pipeline([
    ('preprocessor', preprocessor_lr),
    ('log_reg', LogisticRegression())
])

In [22]:
def objective_lr(trial):
    solver = trial.suggest_categorical('solver', ['liblinear', 'lbfgs', 'saga'])
    if solver == 'lbfgs':
        penalty = 'l2'  # lbfgs só suporta 'l2'
    else:
        penalty = trial.suggest_categorical('penalty', ['l2', 'l1'])  # liblinear e saga suportam 'l1' e 'l2'
    C = trial.suggest_float('C', 0.01, 10.0)
    max_iter = trial.suggest_int('max_iter', 100, 1000)
    pipeline = Pipeline([
    ('preprocessor', preprocessor_lr),
    ('log_reg', LogisticRegression())
    ])
    pipeline.set_params(log_reg__solver=solver, log_reg__penalty=penalty, log_reg__C=C, log_reg__max_iter=max_iter)
    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_validation)

    score = f1_score(y_validation, y_pred)

    return score

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=100, n_jobs=-1)

print(study_lr.best_params)




[I 2024-11-23 19:52:37,154] A new study created in memory with name: no-name-1891cd0c-bedd-4443-9c08-bce3e3a2b063
[I 2024-11-23 19:52:39,098] Trial 0 finished with value: 0.30935251798561153 and parameters: {'solver': 'saga', 'penalty': 'l2', 'C': 1.8144893835258327, 'max_iter': 875}. Best is trial 0 with value: 0.30935251798561153.
[I 2024-11-23 19:52:39,200] Trial 2 finished with value: 0.30824372759856633 and parameters: {'solver': 'lbfgs', 'C': 4.614272775123134, 'max_iter': 464}. Best is trial 0 with value: 0.30935251798561153.
[I 2024-11-23 19:52:39,296] Trial 3 finished with value: 0.3104693140794224 and parameters: {'solver': 'liblinear', 'penalty': 'l2', 'C': 1.051624740348695, 'max_iter': 345}. Best is trial 3 with value: 0.3104693140794224.
[I 2024-11-23 19:52:39,386] Trial 4 finished with value: 0.30824372759856633 and parameters: {'solver': 'lbfgs', 'C': 7.280785660826048, 'max_iter': 486}. Best is trial 3 with value: 0.3104693140794224.
[I 2024-11-23 19:52:39,513] Trial 5

{'solver': 'liblinear', 'penalty': 'l2', 'C': 1.051624740348695, 'max_iter': 345}


Melhor Hiper-parâmetros: 'solver': 'liblinear', 'penalty': 'l2', 'C': 1.051624740348695, 'max_iter': 345

In [ ]:
score = np.zeros(30)
acc = np.zeros(30)
rec = np.zeros(30)
prec = np.zeros(30)
score_train = np.zeros(30)
acc_train = np.zeros(30)
rec_train = np.zeros(30)
prec_train = np.zeros(30)

for i in range(30):
  df = data.sample(frac=1, random_state=42)
  x = df.drop(columns=['Diabetes_binary'])
  y = df['Diabetes_binary']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10, stratify=y)

  pipeline = Pipeline([
    ('preprocessor', preprocessor_lr),
    ('log_reg', LogisticRegression())
  ])

  pipeline.set_params(log_reg__solver='liblinear', log_reg__penalty='l2', log_reg__C=1.051624740348695, log_reg__max_iter=345)
  pipeline.fit(x_train, y_train)
  y_train_pred = pipeline.predict(x_train)
  y_pred = pipeline.predict(x_test)
  score[i] = f1_score(y_test, y_pred)
  acc[i] = accuracy_score(y_test, y_pred)
  rec[i] = recall_score(y_test, y_pred)
  prec[i] = precision_score(y_test, y_pred)

  score_train[i] = f1_score(y_train, y_train_pred)
  acc_train[i] = accuracy_score(y_train, y_train_pred)
  rec_train[i] = recall_score(y_train, y_train_pred)
  prec_train[i] = precision_score(y_train, y_train_pred)


print("MÉTRICAS DE TREINO")
print("F1_SCORE: \n")
print(f" Média -> {score_train.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc_train.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec_train.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec_train.mean()}")
print(f" Desvio Padrão -> {prec_train.std()}")
print("\n\n---------------------\n\n")
print("MÉTRICAS DE TESTE")
print("F1_SCORE: \n")
print(f" Média -> {score.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec.mean()}")
print(f" Desvio Padrão -> {prec.std()}")

MÉTRICAS DE TREINO
F1_SCORE: 

 Média -> 0.8161290322580645
 Desvio Padrão -> 1.1102230246251565e-16

ACCURACY: 

 Média -> 0.7982300884955753
 Desvio Padrão -> 2.220446049250313e-16

RECALL: 

 Média -> 0.8461538461538463
 Desvio Padrão -> 1.1102230246251565e-16

PRECISION: 

 Média -> 0.7881619937694705
 Desvio Padrão -> 1.1102230246251565e-16


---------------------


MÉTRICAS DE TESTE
F1_SCORE: 

 Média -> 0.7066666666666668
 Desvio Padrão -> 1.1102230246251565e-16

ACCURACY: 

 Média -> 0.6901408450704227
 Desvio Padrão -> 2.220446049250313e-16

RECALL: 

 Média -> 0.7066666666666668
 Desvio Padrão -> 1.1102230246251565e-16

PRECISION: 

 Média -> 0.7066666666666668
 Desvio Padrão -> 1.1102230246251565e-16


## SVM

In [24]:
preprocessor_svm = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_columns),
        ('cat', OneHotEncoder(), categorical_columns),
        ('bin', 'passthrough', binary_columns)
    ])

svm_pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])


In [ ]:
def objective_svm(trial):
    pipeline = Pipeline([
        ('preprocessor', preprocessor_svm),
        ('svm', SVC())
    ])


    C = trial.suggest_float('C', 0.001, 1000, log =True)
    gamma = trial.suggest_float('gamma', 0.0001, 10, log = True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])

    pipeline.set_params(svm__C = C, svm__gamma = gamma, svm__kernel = kernel)
    pipeline.fit(x_train, y_train)

    y_pred = pipeline.predict(x_validation)
    score = f1_score(y_validation, y_pred)
    return score



study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=100, n_jobs=-1)


study_svm.best_params

[I 2024-11-23 19:58:13,436] A new study created in memory with name: no-name-51192af0-0479-493b-8df6-3d5b60e0cf44
[I 2024-11-23 19:58:15,443] Trial 1 finished with value: 0.0 and parameters: {'C': 0.22735479490946683, 'gamma': 0.00844163735620407, 'kernel': 'poly'}. Best is trial 1 with value: 0.0.
[I 2024-11-23 19:58:19,028] Trial 2 finished with value: 0.0 and parameters: {'C': 0.031021005264304347, 'gamma': 0.10470680909686261, 'kernel': 'poly'}. Best is trial 1 with value: 0.0.
[I 2024-11-23 19:58:21,551] Trial 3 finished with value: 0.0 and parameters: {'C': 1.0893027338860712, 'gamma': 0.002648728696249403, 'kernel': 'poly'}. Best is trial 1 with value: 0.0.
[I 2024-11-23 19:58:24,575] Trial 4 finished with value: 0.0 and parameters: {'C': 0.1930539506310708, 'gamma': 0.00026267176680971145, 'kernel': 'linear'}. Best is trial 1 with value: 0.0.
[I 2024-11-23 19:58:27,392] Trial 5 finished with value: 0.0 and parameters: {'C': 0.004194158306959299, 'gamma': 0.018063168905382534, '

'C': 165.95337664088618, 'gamma': 0.024468806409998854, 'kernel': 'sigmoid'

In [ ]:
score = np.zeros(30)
acc = np.zeros(30)
rec = np.zeros(30)
prec = np.zeros(30)
score_train = np.zeros(30)
acc_train = np.zeros(30)
rec_train = np.zeros(30)
prec_train = np.zeros(30)

for i in range(30):
  df = data.sample(frac=1, random_state=42)
  x = df.drop(columns=['Diabetes_binary'])
  y = df['Diabetes_binary']
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10, stratify=y)

  pipeline = Pipeline([
    ('preprocessor', preprocessor_svm),
    ('svm', SVC())
  ])
  C = 165.95337664088618
  gamma = 0.024468806409998854
  kernel = 'sigmoid'
  pipeline.set_params(svm__C = C, svm__gamma = gamma, svm__kernel = kernel)
  pipeline.fit(x_train, y_train)


  y_train_pred = pipeline.predict(x_train)
  y_pred = pipeline.predict(x_test)
  score[i] = f1_score(y_test, y_pred)
  acc[i] = accuracy_score(y_test, y_pred)
  rec[i] = recall_score(y_test, y_pred)
  prec[i] = precision_score(y_test, y_pred)

  score_train[i] = f1_score(y_train, y_train_pred)
  acc_train[i] = accuracy_score(y_train, y_train_pred)
  rec_train[i] = recall_score(y_train, y_train_pred)
  prec_train[i] = precision_score(y_train, y_train_pred)


print("MÉTRICAS DE TREINO")
print("F1_SCORE: \n")
print(f" Média -> {score_train.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc_train.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec_train.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec_train.mean()}")
print(f" Desvio Padrão -> {prec_train.std()}")
print("\n\n---------------------\n\n")
print("MÉTRICAS DE TESTE")
print("F1_SCORE: \n")
print(f" Média -> {score.mean()}")
print(f" Desvio Padrão -> {score.std()}")
print("\nACCURACY: \n")
print(f" Média -> {acc.mean()}")
print(f" Desvio Padrão -> {acc.std()}")
print("\nRECALL: \n")
print(f" Média -> {rec.mean()}")
print(f" Desvio Padrão -> {rec.std()}")
print("\nPRECISION: \n")
print(f" Média -> {prec.mean()}")
print(f" Desvio Padrão -> {prec.std()}")